In [1]:
from dask.distributed import Client, LocalCluster
import logging

cluster = LocalCluster(
    n_workers=10,
    threads_per_worker=8,
    silence_logs=logging.DEBUG
)

client = Client(cluster, heartbeat_interval=10000)
print(client.dashboard_link)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:     tcp://127.0.0.1:42743
distributed.scheduler - INFO -   dashboard at:            127.0.0.1:8787
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:46317'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:36407'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:45625'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:35629'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:41721'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:33043'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:36457'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:42537'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:45267'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:34675'
distributed.scheduler - INFO - Register worker <Worker '

http://127.0.0.1:8787/status


In [2]:
cluster.scheduler_address

'tcp://127.0.0.1:42743'

In [3]:
import afqinsight as afqi
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import pandas as pd
import pickle
import seaborn as sns

from datetime import datetime

from sklearn.base import clone
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, roc_auc_score

from skopt import BayesSearchCV
from skopt.plots import plot_convergence, plot_objective, plot_evaluations

print(afqi.__version__)

0.2.9.dev460469908


In [4]:
X, y, groups, columns, subjects, classes = afqi.load_afq_data(
    "../data/raw/als_data",
    target_cols=["class"],
    label_encode_cols=["class"],
)

In [5]:
label_sets = afqi.multicol2sets(pd.MultiIndex.from_tuples(columns, names=["metric", "tractID", "nodeID"]))

In [6]:
pyafq_bundles = [
    c for c in columns
    if c[1] not in ["Right Cingulum Hippocampus", "Left Cingulum Hippocampus"]
]
pyafq_bundles = [
    [c] for c in np.unique([col[1] for col in pyafq_bundles])
]

In [7]:
X_pyafq_bundles = afqi.select_groups(
    X,
    pyafq_bundles,
    label_sets
)

In [8]:
print(X.shape)
print(X_pyafq_bundles.shape)
print(len(label_sets))

(48, 16000)
(48, 14400)
16000


In [9]:
columns = [
    c for c in columns
    if c[1] not in ["Right Cingulum Hippocampus", "Left Cingulum Hippocampus"]
]
label_sets = afqi.multicol2sets(pd.MultiIndex.from_tuples(columns, names=["metric", "tractID", "nodeID"]))

X_md_fa = afqi.select_groups(
    X_pyafq_bundles,
    [["fa"], ["md"]],
    label_sets
)

In [10]:
print(X.shape)
print(X_pyafq_bundles.shape)
print(X_md_fa.shape)

(48, 16000)
(48, 14400)
(48, 3600)


In [11]:
groups_md_fa = groups[:36]

In [12]:
def get_cv_results(n_repeats=5, n_splits=10,
                   power_transformer=False, 
                   shuffle=False,
                   ensembler=None,
                   n_estimators=10,
                   trim_nodes=0,
                   square_features=False):
    if shuffle:
        rng = np.random.default_rng()
        y_fit = rng.permutation(y)
    else:
        y_fit = np.copy(y)

    if trim_nodes > 0:
        grp_mask = np.zeros_like(groups_md_fa[0], dtype=bool)
        grp_mask[trim_nodes:-trim_nodes] = True
        X_mask = np.concatenate([grp_mask] * len(groups_md_fa))

        groups_trim = []
        start_idx = 0
        
        for grp in groups_md_fa:
            stop_idx = start_idx + len(grp) - 2 * trim_nodes
            groups_trim.append(np.arange(start_idx, stop_idx))
            start_idx += len(grp) - 2 * trim_nodes
            
        X_trim = X_md_fa[:, X_mask]
    elif trim_nodes == 0:
        groups_trim = [grp for grp in groups_md_fa]
        X_trim = np.copy(X_md_fa)
    else:
        raise ValueError("trim_nodes must be non-negative.")
        
    if square_features:
        _n_samples, _n_features = X_trim.shape
        X_trim = np.hstack([X_trim, np.square(X_trim)])
        groups_trim = [np.concatenate([g, g + _n_features]) for g in groups_trim]

    cv = RepeatedStratifiedKFold(
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=1729
    )

    cv_results = {}
    pipe_skopt = afqi.pipeline.make_base_afq_pipeline(
        imputer_kwargs={"strategy": "median"},
        power_transformer=power_transformer,
        scaler="standard",
        estimator=LogisticRegressionCV,
        estimator_kwargs={
            "verbose": 0,
            "Cs": 50,
            "penalty": "l1",
            "cv": 3,
            "n_jobs": 10,
            "solver": "saga",
            "max_iter": 500,
        },
        verbose=0,
        ensemble_meta_estimator=ensembler,
        ensemble_meta_estimator_kwargs={
            "n_estimators": n_estimators,
            "n_jobs": 1,
            "oob_score": True,
            "random_state": 1729,
        },
    )
    
    for cv_idx, (train_idx, test_idx) in enumerate(cv.split(X_trim, y_fit)):
        start = datetime.now()

        X_train, X_test = X_trim[train_idx], X_trim[test_idx]
        y_train, y_test = y_fit[train_idx], y_fit[test_idx]

        with joblib.parallel_backend("dask"):
            pipe_skopt.fit(X_train, y_train)

        cv_results[cv_idx] = {
            "pipeline": pipe_skopt,
            "train_idx": train_idx,
            "test_idx": test_idx,
            "y_prob": pipe_skopt.predict_proba(X_test)[:, 1],
            "y_pred": pipe_skopt.predict(X_test),
            "y_true": y_test,
            "test_accuracy": accuracy_score(y_test, pipe_skopt.predict(X_test)),
            "train_accuracy": accuracy_score(y_train, pipe_skopt.predict(X_train)),
            "test_roc_auc": roc_auc_score(y_test, pipe_skopt.predict(X_test)),
            "train_roc_auc": roc_auc_score(y_train, pipe_skopt.predict(X_train)),
            "coefs": [
                est.coef_ for est
                in pipe_skopt.named_steps["estimate"].estimators_
            ],
            "C": [
                est.C_ for est
                in pipe_skopt.named_steps["estimate"].estimators_
            ],
        }
        
        if ensembler is None:
            cv_results[cv_idx]["optimizer"] = pipe_skopt.named_steps["estimate"].bayes_optimizer_

        print(f"CV index [{cv_idx:3d}], Elapsed time: ", datetime.now() - start)
        
    return cv_results, y_fit

In [13]:
results = {}

results[f"bagging_pure_lasso_trim0"] = get_cv_results(
    n_splits=10, n_repeats=1, power_transformer=False,
    ensembler="serial-bagging", shuffle=False,
    n_estimators=20,
    trim_nodes=0, square_features=False
)

distributed.scheduler - INFO - Receive client connection: Client-worker-345aee68-6104-11eb-9fd0-42010a8a0002
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-345cdb58-6104-11eb-9fd0-42010a8a0002
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-345b1f74-6104-11eb-9fd0-42010a8a0002
distributed.core - INFO - Starting established connection
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
distributed.scheduler - INFO - Receive client connection: Client-worker-3

CV index [  0], Elapsed time:  0:02:07.728711


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
distributed.scheduler - INFO - Receive client connection: Client-worker-847a8392-6104-11eb-9fc2-ab5786962b1c
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-84790bfa-6104-11eb-9fc2-42010a8a0002
distributed.core - INFO - Starting established connection
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
distributed.scheduler - INF

CV index [  1], Elapsed time:  0:02:05.721106


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

...

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

CV index [  8], Elapsed time:  0:02:08.191275


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

CV index [  9], Elapsed time:  0:02:07.814020


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [17]:
with open("als_classify_pure_lasso.pkl", "wb") as fp:
    pickle.dump(results, fp)

In [18]:
results.keys()

dict_keys(['bagging_pure_lasso_trim0'])

In [19]:
for key, res in results.items():
    test_accuracies = [cvr["test_accuracy"] for cvr in res[0].values()]
    train_accuracies = [cvr["train_accuracy"] for cvr in res[0].values()]
    test_auc = [cvr["test_roc_auc"] for cvr in res[0].values()]
    train_auc = [cvr["train_roc_auc"] for cvr in res[0].values()]
    
    print(key, "test  acc", np.mean(test_accuracies))
    print(key, "train acc", np.mean(train_accuracies))
    print(key, "test  auc", np.mean(test_auc))
    print(key, "train auc", np.mean(train_auc))
    print()

bagging_pure_lasso_trim0 test  acc 0.8
bagging_pure_lasso_trim0 train acc 0.9930232558139535
bagging_pure_lasso_trim0 test  auc 0.8083333333333332
bagging_pure_lasso_trim0 train auc 0.9930735930735931

